# SkyGeni Sales Intelligence - Comprehensive EDA

**Author**: Vivek Rajbansh  
**Date**: February 9, 2026  
**Purpose**: Deep exploratory data analysis to identify root causes of win rate decline

## Table of Contents
1. [Data Loading & Quality Assessment](#data-quality)
2. [Univariate Analysis](#univariate)
3. [Temporal Analysis](#temporal)
4. [Segmentation Analysis](#segmentation)
5. [Performance Analysis](#performance)
6. [Deal Lifecycle Analysis](#lifecycle)
7. [Key Insights Summary](#insights)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Set visualization style
sns.set_style('whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 7)
plt.rcParams['font.size'] = 11

print('Libraries imported successfully')

## 1. Data Loading & Quality Assessment <a id='data-quality'></a>

First, we load the data and perform comprehensive quality checks.

In [ ]:
# Load dataset
df = pd.read_csv('../data/sales_data.csv')

print(f"Dataset Shape: {df.shape[0]:,} deals × {df.shape[1]} features")
print(f"\nDate Range: {df['created_date'].min()} to {df['created_date'].max()}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data types and basic info
print("Data Types:")
print(df.dtypes)
print("\nBasic Info:")
df.info()

In [ ]:
# Missing value analysis
missing = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Pct': (df.isnull().sum() / len(df) * 100).round(2)
})
missing = missing[missing['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing) > 0:
    print("Missing Values:")
    print(missing)
else:
    print(" No missing values detected")

In [ ]:
# Check for duplicates
duplicates = df.duplicated(subset=['deal_id']).sum()
print(f"Duplicate deal_ids: {duplicates}")

if duplicates == 0:
    print(" No duplicates found")
else:
    print(f" {duplicates} duplicate deals found - need investigation")

In [ ]:
# Convert dates and create time features
df['created_date'] = pd.to_datetime(df['created_date'])
df['closed_date'] = pd.to_datetime(df['closed_date'])

# Create time-based features
df['created_year'] = df['created_date'].dt.year
df['created_quarter'] = df['created_date'].dt.to_period('Q').astype(str)
df['created_month'] = df['created_date'].dt.to_period('M').astype(str)
df['closed_year'] = df['closed_date'].dt.year
df['closed_quarter'] = df['closed_date'].dt.to_period('Q').astype(str)
df['closed_month'] = df['closed_date'].dt.to_period('M').astype(str)

# Binary outcome
df['won'] = (df['outcome'] == 'Won').astype(int)

print(" Date features created successfully")
print(f"\nQuarters in dataset: {sorted(df['closed_quarter'].unique())}")

In [ ]:
# Logical consistency checks
print("Logical Consistency Checks:")
print("="*60)

# Check 1: Closed date should be >= Created date
invalid_dates = (df['closed_date'] < df['created_date']).sum()
print(f"Deals with closed_date < created_date: {invalid_dates}")

# Check 2: Sales cycle should match date difference
df['calculated_cycle'] = (df['closed_date'] - df['created_date']).dt.days
cycle_mismatch = (df['calculated_cycle'] != df['sales_cycle_days']).sum()
print(f"Deals with sales_cycle mismatch: {cycle_mismatch}")

# Check 3: Deal amounts should be positive
negative_amounts = (df['deal_amount'] <= 0).sum()
print(f"Deals with non-positive amounts: {negative_amounts}")

# Check 4: Outcome should be Won or Lost
invalid_outcomes = (~df['outcome'].isin(['Won', 'Lost'])).sum()
print(f"Deals with invalid outcomes: {invalid_outcomes}")

if invalid_dates + cycle_mismatch + negative_amounts + invalid_outcomes == 0:
    print("\n All logical consistency checks passed")
else:
    print("\n Some consistency issues detected")

In [ ]:
# Outlier detection for numerical features
print("Outlier Analysis:")
print("="*60)

def detect_outliers(series, name):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = ((series < lower_bound) | (series > upper_bound)).sum()
    outlier_pct = (outliers / len(series) * 100)
    print(f"{name}: {outliers} outliers ({outlier_pct:.1f}%)")
    return outliers

detect_outliers(df['deal_amount'], 'Deal Amount')
detect_outliers(df['sales_cycle_days'], 'Sales Cycle Days')

print("\nNote: Outliers are not necessarily errors - they may represent legitimate edge cases")

## 2. Univariate Analysis <a id='univariate'></a>

Analyzing each variable individually to understand distributions.

In [ ]:
# Overall win rate
overall_wr = df['won'].mean()
total_wins = df['won'].sum()
total_losses = (~df['won']).sum()

print(f"Overall Win Rate: {overall_wr:.1%}")
print(f"Total Wins: {total_wins:,}")
print(f"Total Losses: {total_losses:,}")

# Visualization
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Pie chart
ax[0].pie([total_wins, total_losses], labels=['Won', 'Lost'], autopct='%1.1f%%',
          colors=['#2ecc71', '#e74c3c'], startangle=90)
ax[0].set_title('Overall Deal Outcomes', fontsize=14, fontweight='bold')

# Bar chart
outcomes = df['outcome'].value_counts()
ax[1].bar(outcomes.index, outcomes.values, color=['#2ecc71', '#e74c3c'])
ax[1].set_title('Deal Outcome Distribution', fontsize=14, fontweight='bold')
ax[1].set_ylabel('Count')
for i, v in enumerate(outcomes.values):
    ax[1].text(i, v + 50, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualizations/01_overall_outcomes.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Deal amount distribution
fig, ax = plt.subplots(1, 3, figsize=(18, 5))

# Histogram
ax[0].hist(df['deal_amount'], bins=50, color='steelblue', edgecolor='black', alpha=0.7)
ax[0].set_title('Deal Amount Distribution', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Deal Amount ($)')
ax[0].set_ylabel('Frequency')
ax[0].axvline(df['deal_amount'].mean(), color='red', linestyle='--', label=f"Mean: ${df['deal_amount'].mean():,.0f}")
ax[0].axvline(df['deal_amount'].median(), color='green', linestyle='--', label=f"Median: ${df['deal_amount'].median():,.0f}")
ax[0].legend()

# Box plot
ax[1].boxplot(df['deal_amount'], vert=True)
ax[1].set_title('Deal Amount Box Plot', fontsize=14, fontweight='bold')
ax[1].set_ylabel('Deal Amount ($)')

# Log scale histogram
ax[2].hist(np.log10(df['deal_amount']), bins=50, color='coral', edgecolor='black', alpha=0.7)
ax[2].set_title('Deal Amount Distribution (Log Scale)', fontsize=14, fontweight='bold')
ax[2].set_xlabel('Log10(Deal Amount)')
ax[2].set_ylabel('Frequency')

plt.tight_layout()
plt.savefig('../visualizations/02_deal_amount_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Statistics
print("Deal Amount Statistics:")
print(df['deal_amount'].describe())

In [ ]:
# Sales cycle distribution
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax[0].hist(df['sales_cycle_days'], bins=50, color='purple', edgecolor='black', alpha=0.7)
ax[0].set_title('Sales Cycle Distribution', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Sales Cycle (Days)')
ax[0].set_ylabel('Frequency')
ax[0].axvline(df['sales_cycle_days'].mean(), color='red', linestyle='--', 
              label=f"Mean: {df['sales_cycle_days'].mean():.0f} days")
ax[0].legend()

# Box plot by outcome
df.boxplot(column='sales_cycle_days', by='outcome', ax=ax[1])
ax[1].set_title('Sales Cycle by Outcome', fontsize=14, fontweight='bold')
ax[1].set_xlabel('Outcome')
ax[1].set_ylabel('Sales Cycle (Days)')
plt.suptitle('')

plt.tight_layout()
plt.savefig('../visualizations/03_sales_cycle_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("Sales Cycle Statistics:")
print(df.groupby('outcome')['sales_cycle_days'].describe())

In [ ]:
# Categorical variable distributions
categorical_cols = ['region', 'industry', 'product_type', 'lead_source', 'deal_stage']

fig, axes = plt.subplots(2, 3, figsize=(18, 10))
axes = axes.flatten()

for idx, col in enumerate(categorical_cols):
    value_counts = df[col].value_counts()
    axes[idx].bar(range(len(value_counts)), value_counts.values, color='teal', alpha=0.7)
    axes[idx].set_xticks(range(len(value_counts)))
    axes[idx].set_xticklabels(value_counts.index, rotation=45, ha='right')
    axes[idx].set_title(f'{col.replace("_", " ").title()} Distribution', fontweight='bold')
    axes[idx].set_ylabel('Count')
    
    # Add value labels
    for i, v in enumerate(value_counts.values):
        axes[idx].text(i, v + 20, str(v), ha='center', fontsize=9)

# Remove extra subplot
fig.delaxes(axes[5])

plt.tight_layout()
plt.savefig('../visualizations/04_categorical_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

## 3. Temporal Analysis <a id='temporal'></a>

Analyzing trends over time to identify when the decline began.

In [ ]:
# Quarterly win rate trend
quarterly_stats = df.groupby('closed_quarter').agg({
    'won': ['sum', 'count', 'mean'],
    'deal_amount': 'sum'
})
quarterly_stats.columns = ['Wins', 'Total', 'Win_Rate', 'Total_ACV']
quarterly_stats['Win_Rate_Pct'] = quarterly_stats['Win_Rate'] * 100
quarterly_stats['Avg_Deal_Size'] = quarterly_stats['Total_ACV'] / quarterly_stats['Total']

print("Quarterly Performance:")
print(quarterly_stats[['Total', 'Wins', 'Win_Rate_Pct', 'Avg_Deal_Size']].round(1))

In [ ]:
# Visualization: Quarterly trends
fig, ax = plt.subplots(2, 2, figsize=(16, 10))

quarters = quarterly_stats.index
x_pos = range(len(quarters))

# Win rate trend
ax[0, 0].plot(x_pos, quarterly_stats['Win_Rate_Pct'], marker='o', linewidth=2, markersize=8, color='#3498db')
ax[0, 0].axhline(y=quarterly_stats['Win_Rate_Pct'].mean(), color='red', linestyle='--', label='Average')
ax[0, 0].set_xticks(x_pos)
ax[0, 0].set_xticklabels(quarters, rotation=45, ha='right')
ax[0, 0].set_title('Win Rate Trend by Quarter', fontsize=14, fontweight='bold')
ax[0, 0].set_ylabel('Win Rate (%)')
ax[0, 0].legend()
ax[0, 0].grid(True, alpha=0.3)

# Volume trend
ax[0, 1].bar(x_pos, quarterly_stats['Total'], color='#2ecc71', alpha=0.7)
ax[0, 1].set_xticks(x_pos)
ax[0, 1].set_xticklabels(quarters, rotation=45, ha='right')
ax[0, 1].set_title('Deal Volume by Quarter', fontsize=14, fontweight='bold')
ax[0, 1].set_ylabel('Number of Deals')
for i, v in enumerate(quarterly_stats['Total']):
    ax[0, 1].text(i, v + 20, str(v), ha='center', fontweight='bold')

# Revenue trend
ax[1, 0].bar(x_pos, quarterly_stats['Total_ACV'] / 1000000, color='#9b59b6', alpha=0.7)
ax[1, 0].set_xticks(x_pos)
ax[1, 0].set_xticklabels(quarters, rotation=45, ha='right')
ax[1, 0].set_title('Total ACV by Quarter', fontsize=14, fontweight='bold')
ax[1, 0].set_ylabel('Total ACV ($M)')
for i, v in enumerate(quarterly_stats['Total_ACV'] / 1000000):
    ax[1, 0].text(i, v + 0.2, f'${v:.1f}M', ha='center', fontweight='bold')

# Average deal size trend
ax[1, 1].plot(x_pos, quarterly_stats['Avg_Deal_Size'], marker='s', linewidth=2, markersize=8, color='#e74c3c')
ax[1, 1].set_xticks(x_pos)
ax[1, 1].set_xticklabels(quarters, rotation=45, ha='right')
ax[1, 1].set_title('Average Deal Size by Quarter', fontsize=14, fontweight='bold')
ax[1, 1].set_ylabel('Avg Deal Size ($)')
ax[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/05_quarterly_trends.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Identify last two quarters vs historical
quarters_list = sorted(df['closed_quarter'].unique())
last_two_quarters = quarters_list[-2:]
historical_quarters = quarters_list[:-2]

recent_df = df[df['closed_quarter'].isin(last_two_quarters)]
historical_df = df[df['closed_quarter'].isin(historical_quarters)]

print(f"Last Two Quarters: {last_two_quarters}")
print(f"Historical Quarters: {historical_quarters}")
print(f"\nRecent deals: {len(recent_df):,}")
print(f"Historical deals: {len(historical_df):,}")

recent_wr = recent_df['won'].mean()
historical_wr = historical_df['won'].mean()
decline = (recent_wr - historical_wr) * 100

print(f"\nHistorical Win Rate: {historical_wr:.1%}")
print(f"Recent Win Rate: {recent_wr:.1%}")
print(f"Decline: {decline:+.1f} percentage points")

In [ ]:
# Monthly granularity analysis
monthly_stats = df.groupby('closed_month').agg({
    'won': ['sum', 'count', 'mean']
})
monthly_stats.columns = ['Wins', 'Total', 'Win_Rate']
monthly_stats['Win_Rate_Pct'] = monthly_stats['Win_Rate'] * 100

# Plot monthly win rate with moving average
fig, ax = plt.subplots(figsize=(16, 6))

x_pos = range(len(monthly_stats))
ax.plot(x_pos, monthly_stats['Win_Rate_Pct'], marker='o', linewidth=1, markersize=4, 
        color='#3498db', alpha=0.5, label='Monthly Win Rate')

# 3-month moving average
ma_3 = monthly_stats['Win_Rate_Pct'].rolling(window=3).mean()
ax.plot(x_pos, ma_3, linewidth=3, color='#e74c3c', label='3-Month Moving Avg')

ax.set_xticks(x_pos[::2])  # Show every other month
ax.set_xticklabels(monthly_stats.index[::2], rotation=45, ha='right')
ax.set_title('Monthly Win Rate Trend with Moving Average', fontsize=14, fontweight='bold')
ax.set_ylabel('Win Rate (%)')
ax.set_xlabel('Month')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/06_monthly_win_rate_trend.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Segmentation Analysis <a id='segmentation'></a>

Deep dive into performance across different segments.

In [ ]:
# Function to compare segments
def analyze_segment(segment_col, title):
    """
    Comprehensive segment analysis comparing recent vs historical performance
    """
    # Calculate statistics
    recent_stats = recent_df.groupby(segment_col).agg({
        'won': ['mean', 'count']
    })
    recent_stats.columns = ['Recent_WR', 'Recent_Vol']
    
    hist_stats = historical_df.groupby(segment_col).agg({
        'won': ['mean', 'count']
    })
    hist_stats.columns = ['Hist_WR', 'Hist_Vol']
    
    comparison = pd.DataFrame({
        'Historical_WR': (hist_stats['Hist_WR'] * 100).round(1),
        'Recent_WR': (recent_stats['Recent_WR'] * 100).round(1),
        'Decline_pp': ((recent_stats['Recent_WR'] - hist_stats['Hist_WR']) * 100).round(1),
        'Recent_Volume': recent_stats['Recent_Vol'],
        'Historical_Volume': hist_stats['Hist_Vol']
    }).sort_values('Decline_pp')
    
    # Visualization
    fig, ax = plt.subplots(1, 2, figsize=(16, 6))
    
    # Win rate comparison
    x = np.arange(len(comparison))
    width = 0.35
    
    ax[0].bar(x - width/2, comparison['Historical_WR'], width, label='Historical', color='#3498db', alpha=0.8)
    ax[0].bar(x + width/2, comparison['Recent_WR'], width, label='Recent', color='#e74c3c', alpha=0.8)
    ax[0].set_xlabel(title)
    ax[0].set_ylabel('Win Rate (%)')
    ax[0].set_title(f'Win Rate Comparison: {title}', fontweight='bold')
    ax[0].set_xticks(x)
    ax[0].set_xticklabels(comparison.index, rotation=45, ha='right')
    ax[0].legend()
    ax[0].grid(True, alpha=0.3, axis='y')
    
    # Decline visualization
    colors = ['#e74c3c' if x < 0 else '#2ecc71' for x in comparison['Decline_pp']]
    ax[1].barh(range(len(comparison)), comparison['Decline_pp'], color=colors, alpha=0.8)
    ax[1].set_yticks(range(len(comparison)))
    ax[1].set_yticklabels(comparison.index)
    ax[1].set_xlabel('Win Rate Change (pp)')
    ax[1].set_title(f'Win Rate Decline by {title}', fontweight='bold')
    ax[1].axvline(x=0, color='black', linestyle='-', linewidth=0.8)
    ax[1].grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, v in enumerate(comparison['Decline_pp']):
        ax[1].text(v, i, f' {v:+.1f}pp', va='center', fontweight='bold')
    
    plt.tight_layout()
    filename = f"../visualizations/07_{segment_col}_analysis.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')
    plt.show()
    
    # Print summary
    print(f"\n{title} Analysis:")
    print("="*80)
    print(comparison)
    print(f"\nBiggest Decline: {comparison.index[0]} ({comparison['Decline_pp'].iloc[0]:+.1f}pp)")
    print(f"Biggest Improvement: {comparison.index[-1]} ({comparison['Decline_pp'].iloc[-1]:+.1f}pp)")
    
    return comparison

In [ ]:
# Analyze each segment
region_analysis = analyze_segment('region', 'Region')

In [ ]:
industry_analysis = analyze_segment('industry', 'Industry')

In [ ]:
product_analysis = analyze_segment('product_type', 'Product Type')

In [ ]:
source_analysis = analyze_segment('lead_source', 'Lead Source')

In [ ]:
stage_analysis = analyze_segment('deal_stage', 'Deal Stage')

## 5. Performance Analysis <a id='performance'></a>

Sales rep and deal characteristic analysis.

In [ ]:
# Sales rep performance analysis
rep_performance = df.groupby('sales_rep_id').agg({
    'won': ['sum', 'count', 'mean'],
    'deal_amount': 'sum'
})
rep_performance.columns = ['Wins', 'Total', 'Win_Rate', 'Total_ACV']
rep_performance = rep_performance[rep_performance['Total'] >= 50]  # Min 50 deals
rep_performance['Win_Rate_Pct'] = (rep_performance['Win_Rate'] * 100).round(1)
rep_performance = rep_performance.sort_values('Win_Rate', ascending=False)

print("Top 10 Sales Reps (min 50 deals):")
print(rep_performance.head(10)[['Total', 'Wins', 'Win_Rate_Pct', 'Total_ACV']])

print("\nBottom 10 Sales Reps (min 50 deals):")
print(rep_performance.tail(10)[['Total', 'Wins', 'Win_Rate_Pct', 'Total_ACV']])

In [ ]:
# Rep performance visualization
fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# Win rate distribution
ax[0].hist(rep_performance['Win_Rate_Pct'], bins=15, color='steelblue', edgecolor='black', alpha=0.7)
ax[0].axvline(rep_performance['Win_Rate_Pct'].mean(), color='red', linestyle='--', 
              label=f"Mean: {rep_performance['Win_Rate_Pct'].mean():.1f}%")
ax[0].set_title('Sales Rep Win Rate Distribution', fontsize=14, fontweight='bold')
ax[0].set_xlabel('Win Rate (%)')
ax[0].set_ylabel('Number of Reps')
ax[0].legend()

# Top vs Bottom performers
top_5 = rep_performance.head(5)
bottom_5 = rep_performance.tail(5)

combined = pd.concat([top_5, bottom_5])
colors = ['#2ecc71'] * 5 + ['#e74c3c'] * 5

ax[1].barh(range(len(combined)), combined['Win_Rate_Pct'], color=colors, alpha=0.8)
ax[1].set_yticks(range(len(combined)))
ax[1].set_yticklabels(combined.index)
ax[1].set_xlabel('Win Rate (%)')
ax[1].set_title('Top 5 vs Bottom 5 Reps', fontsize=14, fontweight='bold')
ax[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('../visualizations/08_rep_performance.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Deal size vs win rate analysis
# Create deal size buckets
df['deal_size_bucket'] = pd.cut(df['deal_amount'], 
                                  bins=[0, 10000, 30000, 60000, 100000],
                                  labels=['<$10K', '$10-30K', '$30-60K', '>$60K'])

size_wr = df.groupby('deal_size_bucket').agg({
    'won': ['mean', 'count']
})
size_wr.columns = ['Win_Rate', 'Count']
size_wr['Win_Rate_Pct'] = (size_wr['Win_Rate'] * 100).round(1)

print("Win Rate by Deal Size:")
print(size_wr)

# Visualization
fig, ax = plt.subplots(1, 2, figsize=(14, 5))

ax[0].bar(range(len(size_wr)), size_wr['Win_Rate_Pct'], color='teal', alpha=0.7)
ax[0].set_xticks(range(len(size_wr)))
ax[0].set_xticklabels(size_wr.index)
ax[0].set_title('Win Rate by Deal Size', fontsize=14, fontweight='bold')
ax[0].set_ylabel('Win Rate (%)')
ax[0].set_xlabel('Deal Size Bucket')
for i, v in enumerate(size_wr['Win_Rate_Pct']):
    ax[0].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

ax[1].bar(range(len(size_wr)), size_wr['Count'], color='coral', alpha=0.7)
ax[1].set_xticks(range(len(size_wr)))
ax[1].set_xticklabels(size_wr.index)
ax[1].set_title('Deal Volume by Size', fontsize=14, fontweight='bold')
ax[1].set_ylabel('Number of Deals')
ax[1].set_xlabel('Deal Size Bucket')
for i, v in enumerate(size_wr['Count']):
    ax[1].text(i, v + 20, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualizations/09_deal_size_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Sales cycle vs win rate analysis
df['cycle_bucket'] = pd.cut(df['sales_cycle_days'],
                              bins=[0, 30, 60, 90, 150],
                              labels=['<30d', '30-60d', '60-90d', '>90d'])

cycle_wr = df.groupby('cycle_bucket').agg({
    'won': ['mean', 'count']
})
cycle_wr.columns = ['Win_Rate', 'Count']
cycle_wr['Win_Rate_Pct'] = (cycle_wr['Win_Rate'] * 100).round(1)

print("Win Rate by Sales Cycle Length:")
print(cycle_wr)

# Visualization
fig, ax = plt.subplots(figsize=(10, 6))

ax.bar(range(len(cycle_wr)), cycle_wr['Win_Rate_Pct'], color='purple', alpha=0.7)
ax.set_xticks(range(len(cycle_wr)))
ax.set_xticklabels(cycle_wr.index)
ax.set_title('Win Rate by Sales Cycle Length', fontsize=14, fontweight='bold')
ax.set_ylabel('Win Rate (%)')
ax.set_xlabel('Sales Cycle Bucket')
for i, v in enumerate(cycle_wr['Win_Rate_Pct']):
    ax.text(i, v + 1, f'{v:.1f}%\n(n={cycle_wr["Count"].iloc[i]})', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('../visualizations/10_sales_cycle_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Deal Lifecycle Analysis <a id='lifecycle'></a>

Analyzing the sales funnel and stage conversions.

In [ ]:
# Stage distribution and win rates
stage_stats = df.groupby('deal_stage').agg({
    'won': ['mean', 'count']
})
stage_stats.columns = ['Win_Rate', 'Count']
stage_stats['Win_Rate_Pct'] = (stage_stats['Win_Rate'] * 100).round(1)
stage_stats = stage_stats.sort_values('Win_Rate', ascending=False)

print("Win Rate by Deal Stage:")
print(stage_stats)

# Visualization: Funnel
fig, ax = plt.subplots(figsize=(12, 8))

# Create funnel visualization
stages_ordered = ['Qualified', 'Demo', 'Proposal', 'Negotiation', 'Closed']
stage_counts = [df[df['deal_stage'] == stage].shape[0] for stage in stages_ordered]
stage_wrs = [df[df['deal_stage'] == stage]['won'].mean() * 100 for stage in stages_ordered]

colors_funnel = plt.cm.Blues(np.linspace(0.4, 0.9, len(stages_ordered)))

for i, (stage, count, wr) in enumerate(zip(stages_ordered, stage_counts, stage_wrs)):
    width = count / max(stage_counts)
    ax.barh(i, width, height=0.8, color=colors_funnel[i], edgecolor='black', linewidth=2)
    ax.text(width/2, i, f'{stage}\n{count} deals\n{wr:.1f}% WR', 
            ha='center', va='center', fontweight='bold', fontsize=11)

ax.set_yticks(range(len(stages_ordered)))
ax.set_yticklabels([])
ax.set_xlabel('Relative Volume', fontsize=12)
ax.set_title('Sales Funnel by Stage', fontsize=14, fontweight='bold')
ax.set_xlim(0, 1.1)

plt.tight_layout()
plt.savefig('../visualizations/11_sales_funnel.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Key Insights Summary <a id='insights'></a>

Consolidating all findings into actionable insights.

In [ ]:
# Create comprehensive insights summary
insights_summary = f"""
KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS
{'='*80}

1. OVERALL PERFORMANCE
   - Historical Win Rate: {historical_wr:.1%}
   - Recent Win Rate: {recent_wr:.1%}
   - Decline: {decline:+.1f} percentage points
   - Pipeline Volume Change: {((len(recent_df)/2) / (len(historical_df)/len(historical_quarters)) - 1) * 100:+.1f}%

2. BIGGEST PROBLEM AREAS (Win Rate Decline)
   - Lead Source: {source_analysis.index[0]} ({source_analysis['Decline_pp'].iloc[0]:+.1f}pp)
   - Deal Stage: {stage_analysis.index[0]} ({stage_analysis['Decline_pp'].iloc[0]:+.1f}pp)
   - Industry: {industry_analysis.index[0]} ({industry_analysis['Decline_pp'].iloc[0]:+.1f}pp)

3. BRIGHT SPOTS (Win Rate Improvement)
   - Lead Source: {source_analysis.index[-1]} ({source_analysis['Decline_pp'].iloc[-1]:+.1f}pp)
   - Industry: {industry_analysis.index[-1]} ({industry_analysis['Decline_pp'].iloc[-1]:+.1f}pp)

4. SALES CYCLE IMPACT
   - Historical Average: {historical_df['sales_cycle_days'].mean():.0f} days
   - Recent Average: {recent_df['sales_cycle_days'].mean():.0f} days
   - Increase: {((recent_df['sales_cycle_days'].mean() / historical_df['sales_cycle_days'].mean()) - 1) * 100:+.1f}%

5. REP PERFORMANCE VARIANCE
   - Top Rep Win Rate: {rep_performance['Win_Rate_Pct'].iloc[0]:.1f}%
   - Bottom Rep Win Rate: {rep_performance['Win_Rate_Pct'].iloc[-1]:.1f}%
   - Performance Gap: {rep_performance['Win_Rate_Pct'].iloc[0] - rep_performance['Win_Rate_Pct'].iloc[-1]:.1f}pp

6. REVENUE IMPACT
   - Historical Quarterly Revenue: ${historical_df[historical_df['won']==1]['deal_amount'].sum() / len(historical_quarters) / 1000000:.1f}M
   - Recent Quarterly Revenue: ${recent_df[recent_df['won']==1]['deal_amount'].sum() / 2 / 1000000:.1f}M
   - Quarterly Revenue Decline: ${(historical_df[historical_df['won']==1]['deal_amount'].sum() / len(historical_quarters) - recent_df[recent_df['won']==1]['deal_amount'].sum() / 2) / 1000000:.1f}M
"""

print(insights_summary)

# Save insights to file
with open('../docs/02_eda_insights.txt', 'w') as f:
    f.write(insights_summary)

print("\n Insights saved to docs/02_eda_insights.txt")

In [ ]:
# Save processed dataframes for next steps
df.to_csv('../data/processed_sales_data.csv', index=False)
recent_df.to_csv('../data/recent_deals.csv', index=False)
historical_df.to_csv('../data/historical_deals.csv', index=False)

print(" Processed data saved for model development")
print("\nNext Steps:")
print("1. Calculate custom metrics")
print("2. Build win rate driver analysis model")
print("3. Create executive summary")